<a href="https://colab.research.google.com/github/gogzicole/Hamoye-Data-science/blob/master/HAMOYE_STAGE_C_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [3]:
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.isnull().sum()

**counts the total number of values in target variable**

In [4]:
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1B       16
1A       16
Name: QScore, dtype: int64

In [5]:
df = df.dropna()
df.head()
df['QScore'].value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

**converting to a binary classificatio problem by joining class 2A and 1A**

In [6]:
df['QScore'] = df['QScore'].replace(['1A'], '2A')
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [7]:
df_2A = df[df.QScore=='2A']
df_2A.count()


country           240
year              240
country_code      240
record            240
crop_land         240
grazing_land      240
forest_land       240
fishing_ground    240
built_up_land     240
carbon            240
total             240
QScore            240
dtype: int64

In [8]:
df_3A = df[df.QScore=='3A'].sample(350)
df_3A.count()

country           350
year              350
country_code      350
record            350
crop_land         350
grazing_land      350
forest_land       350
fishing_ground    350
built_up_land     350
carbon            350
total             350
QScore            350
dtype: int64

In [18]:
data_df = df_2A.append(df_3A)
data_df.count()

country           590
year              590
country_code      590
record            590
crop_land         590
grazing_land      590
forest_land       590
fishing_ground    590
built_up_land     590
carbon            590
total             590
QScore            590
dtype: int64

**shuffuling the dataset to randomize distribution of 2A and 3A**

In [19]:
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape
data_df['QScore'].value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

**dropping the first 3 columns and re-assigning QScore to y and the remaining data set to x**

In [20]:
data_df = data_df.drop(columns=['country_code', 'country', 'year'])
x = data_df.drop(columns='QScore')
y = data_df['QScore']


**splitting  data into test and split**

In [50]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)
y_train.value_counts()

3A    235
2A    178
Name: QScore, dtype: int64

**now we must encode categorical data**

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record)
x_test.record = encoder.transform(x_test.record)

**if there is still an inbalance on the class distribution, for this we must use SMOTE to handle this, but only on the training data**

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_sample(x_train, y_train)
x_train_balanced = DataFrame(x_train_balanced, columns=x_train.columns)

x_train_balanced

**now we perform feature scaling on the dataset**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record'] = x_train_balanced['record']
normalised_train_df

In [54]:
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']


**Logistics regression**

In [58]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

**cross validation and accuracy**

In [59]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv=5, scoring='f1_macro')
scores

array([0.5106383 , 0.41614907, 0.45503886, 0.51342241, 0.50968703])

**k-Fold**

In [60]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix

In [61]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(normalised_train_df) 
f1_scores = []
#run for every split
for train_index, test_index in kf.split(normalised_train_df):
    x_train, x_test = normalised_train_df.iloc[train_index],normalised_train_df.iloc[test_index]
    y_train, y_test = y_balanced[train_index],y_balanced[test_index]
    model = LogisticRegression().fit(x_train, y_train)
#save result to list
    f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test),pos_label='2A')*100)

**stratified KFold cross validation**

In [62]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
f1_scores = []
#run for every split
for train_index, test_index in skf.split(normalised_train_df, y_balanced):
    x_train, x_test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]
    y_train, y_test  = y_balanced[train_index], y_balanced[test_index]
    model = LogisticRegression().fit(x_train, y_train)
#save result to list
    f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A'))

**leave one out cross validation (loocv)**

In [63]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo,scoring='f1_macro')
average_score = scores.mean() * 100

**Evaluation Metrics Used in Measuring The Performance of Classification Models**

**1. confusion matrix**

In [ ]:
new_predictions = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=['2A', '3A'])

**2. Accuracy**

In [ ]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
print('Accuracy: {}'.format(round(accuracy*100), 2)) 

**3. precission**

In [ ]:
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('Precision: {}'.format(round(precision*100), 2))

**4. Recall**

In [ ]:
recall = recall_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('Recall: {}'.format(round(recall*100), 2))

**5. f1_score**

In [ ]:
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('F1: {}'.format(round(f1*100), 2))